# 预读取

In [1]:
import h5py
import cv2
import warp_norm
import matplotlib
# import sys
# sys.path.append("./FaceAlignment")
# import face_alignment
from skimage import io
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from model import gaze_network
from torchvision import transforms
import pickle
import pandas as pd
import copy
import os
from pt_module import StNet,StRefine
from ipdb import set_trace as st
import gaze_normalize

cam_chen = '/home/hgh/hghData/Datasets/camChen.xml'  # this is camera calibration information file obtained with OpenCV
fs_chen = cv2.FileStorage(cam_chen, cv2.FILE_STORAGE_READ)
camera_matrix_chen = fs_chen.getNode('Camera_Matrix').mat() # camera calibration information is used for data normalization
camera_distortion_chen = fs_chen.getNode('Distortion_Coefficients').mat()

cam_tan = '/home/hgh/hghData/Datasets/camTan.xml'  # this is camera calibration information file obtained with OpenCV
fs_tan = cv2.FileStorage(cam_tan, cv2.FILE_STORAGE_READ)
camera_matrix_tan = fs_tan.getNode('Camera_Matrix').mat() # camera calibration information is used for data normalization
camera_distortion_tan = fs_tan.getNode('Distortion_Coefficients').mat()

pixel_scale_tan = np.array([0.202, 0.224])
pixel_scale_chen = np.array([0.22, 0.235])

## 对部分数据集做处理并施加调整

In [2]:
image_folder_path = '/home/hgh/hghData/Datasets2/part'
save_dir = '/home/hgh/hghData/Datasets2/part_preprocess'
csv_file_path = '/home/hgh/hghData/Datasets2/coordinate.csv'
df = pd.read_csv(csv_file_path, header=None)
preds = gaze_normalize.xmodel()

org_data = []
#遍历图像文件夹
for filename in sorted(os.listdir(image_folder_path), key=lambda x: int(os.path.splitext(x)[0])):
    if filename.endswith(".jpg"):
        # 构建图像文件的完整路径
        image_path = os.path.join(image_folder_path, filename)
        print(image_path)
        try:
            row = df.iloc[int(os.path.splitext(filename)[0]) - 1].tolist()
            label = (int(row[3]),int(row[4]))
        except:
            st()
        print(label)
        gaze_normalize_new = gaze_normalize.GazeNormalize(filename,label,camera_matrix_tan,camera_distortion_tan,preds)
        save_path = os.path.join(save_dir, f'{filename}')
        warp_image = gaze_normalize_new.norm(image_folder_path)
        # if gaze_normalize_new.err == False:
        #     cv2.imwrite(save_path, warp_image)
        org_data.append(gaze_normalize_new)

/home/hgh/hghData/Datasets2/part/2001.jpg
(829, 44)
/home/hgh/hghData/Datasets2/part/2002.jpg
(929, 63)
/home/hgh/hghData/Datasets2/part/2003.jpg
(1302, 303)
/home/hgh/hghData/Datasets2/part/2004.jpg
(68, 321)
/home/hgh/hghData/Datasets2/part/2005.jpg
(178, 246)
/home/hgh/hghData/Datasets2/part/2006.jpg
(631, 467)
/home/hgh/hghData/Datasets2/part/2007.jpg
(745, 633)
/home/hgh/hghData/Datasets2/part/2008.jpg
(29, 372)
/home/hgh/hghData/Datasets2/part/2009.jpg
(756, 171)
/home/hgh/hghData/Datasets2/part/2010.jpg
(86, 635)
/home/hgh/hghData/Datasets2/part/2101.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2102.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2103.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2104.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2105.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2106.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2107.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2108.jpg
(-1, -1)
/home/hgh/hghData/Datasets2/part/2109.jpg
(-1, -1)
/home/hgh/hghDa

In [3]:
import math
# # 确保欧拉角转换流程无误
# for i in range(20):
#     print('hr:',org_data[i].hr)
#     print('euler:',warp_norm.hr_to_pitchyaw(org_data[i].hr))
#     euler = warp_norm.hr_to_pitchyaw(org_data[i].hr) 
#     print('re hr:', warp_norm.pitchyaw_to_hr(euler))
#     print('-----')

# 操作流程
# 对hr作调整
# 对pitch作调整, epi 0.05deg
condit_dict = []
epi  = 0.05
face_model_load = np.loadtxt('./modules/face_model.txt')  # Generic face model with 3D facial landmarks
landmark_use = [20, 23, 26, 29, 15, 19]  # we use eye corners and nose conners
face_model = face_model_load[landmark_use, :]
print('load gaze estimator')
model_path = './ckpt/epoch_24_ckpt.pth.tar'
model = gaze_network()
model.cuda()
pre_trained_model_path = model_path
if not os.path.isfile(pre_trained_model_path):
    print('the pre-trained gaze estimation model does not exist.')
    exit(0)
else:
    print('load the pre-trained model: ', pre_trained_model_path)
ckpt = torch.load(pre_trained_model_path)
model.load_state_dict(ckpt['model_state'], strict=True)  # load the pre-trained model
model.eval()  # change it to the evaluation mode

for i in range(10):
    for j in range(-10,11):
        for k in range(3):
            temp = copy.deepcopy(org_data[i])
            euler = warp_norm.hr_to_pitchyaw(temp.hr)
            euler[0,k] += epi*j
            temp.hr = warp_norm.pitchyaw_to_hr(euler)
            image_warp = temp.xtrans(face_model)
            temp.pred(model, image_warp)
            temp.vector_to_screen(pixel_scale_tan)
            condit_dict.append(temp)

load gaze estimator
load the pre-trained model:  ./ckpt/epoch_24_ckpt.pth.tar


In [4]:
org_tan = np.array([800,0])#tan 1600*825
# pred_gc_org[i] = org_tan + pred_gc[i]


## 多因素方差分析

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 合成dataframe
subjects = []
pitch = []
yaw = []
roll = []
preds = []
for data in condit_dict:
    subjects.append(data.image_name)
    euler = warp_norm.hr_to_pitchyaw(data.hr)
    pitch.append(euler[0,0])
    yaw.append(euler[0,1])
    roll.append(euler[0,2])
    preds.append(data.gaze_point+org_tan)

preds = np.asarray(preds)
data = {
    'subject_id': subjects,
    'pitch': pitch,
    'yaw': yaw,
    'roll': roll,
    'prediction_x': preds[:,0],
    'prediction_y': preds[:,1]
}
df = pd.DataFrame(data)
formula = 'prediction_x ~ pitch + yaw + roll'
model_x = smf.mixedlm(formula, data=data, groups=data['subject_id'])
model_y = smf.mixedlm(formula, data=data, groups=data['subject_id'])


result_x = model_x.fit()
result_y = model_y.fit()

# 打印方差分析结果
print(result_x.summary())
print(result_y.summary())


    

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: prediction_x
No. Observations: 630     Method:             REML        
No. Groups:       10      Scale:              525.9987    
Min. group size:  63      Log-Likelihood:     -2896.9324  
Max. group size:  63      Converged:          Yes         
Mean group size:  63.0                                    
----------------------------------------------------------
              Coef.   Std.Err.   z   P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept     562.265   85.147 6.603 0.000 395.380 729.150
pitch          11.400    5.119 2.227 0.026   1.367  21.434
yaw             5.191    5.182 1.002 0.316  -4.965  15.348
roll           24.236    5.229 4.635 0.000  13.988  34.484
Group Var   59603.008 1249.988                            

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: prediction_x
No. Observations: 